# Korg for GALAH

In [1]:
try:
    %matplotlib inline
    %config InlineBackend.figure_format='retina'
except:
    pass

# Korg and Julia
from juliacall import Main as jl
jl.seval("using Korg")
Korg = jl.Korg

# general packages
import time
import numpy as np
from astropy.io import fits
from astropy.table import Table
from scipy.io import readsav

# matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

In [2]:
# Prepare HERMES information
galah_lines = Korg.get_GALAH_DR3_linelist()

In [3]:
def rv_shift(rv_value, wavelength):
    return(wavelength / (1.+rv_value/299792.458))

In [4]:
def get_spectrum(sobject_id):
    
    spectrum = dict()
    spectrum['sobject_id'] = sobject_id
    
    for ccd in [1,2,3,4]:
        
        fits_file = fits.open('observations/'+str(sobject_id)[:6]+'/spectra/com/'+str(sobject_id)+str(ccd)+'.fits')
        spectrum['wave_air_ccd'+str(ccd)] = rv_shift(fits_file[0].header['RVCOM'], fits_file[0].header['CRVAL1'] + fits_file[0].header['CDELT1'] * np.arange(fits_file[0].header['NAXIS1']))
        spectrum['counts_ccd'+str(ccd)] = fits_file[0].data
        spectrum['counts_unc_ccd'+str(ccd)] = fits_file[0].data * fits_file[2].data
        spectrum['flux_norm_ccd'+str(ccd)] = fits_file[1].data
        spectrum['flux_unc_norm_ccd'+str(ccd)] = fits_file[1].data * fits_file[2].data
        
        fits_file.close()
        
    return(spectrum)

spectrum = get_spectrum(sobject_id = 210115002201239)

# ELEMENT ABUNDANCES

In [5]:
stellar_parameters = dict(Teff = 5772, logg = 4.44, m_H = 0.00, vsini = 4.0, vmic=1.06)

In [6]:
spectrum_waveobs = np.concatenate(([[Korg.air_to_vacuum(λ) for λ in spectrum['wave_air_ccd'+str(ccd)]] for ccd in [1,2,3,4]]))
spectrum_flux = np.concatenate(([[f for f in spectrum['flux_norm_ccd'+str(ccd)]] for ccd in [1,2,3,4]]))
spectrum_err = np.concatenate(([[f for f in spectrum['flux_unc_norm_ccd'+str(ccd)]] for ccd in [1,2,3,4]]))

In [7]:
spectrum_s = Table()

o_window = (spectrum_waveobs > 7770) & (spectrum_waveobs < 7780)

spectrum_s['waveobs'] = spectrum_waveobs[o_window]
spectrum_s['flux'] = spectrum_flux[o_window]
spectrum_s['err'] = spectrum_err[o_window]
spectrum_s.write('Vesta_GALAH.csv',overwrite=True)

In [8]:
synthesis_wls = jl.seval('7750:0.01:7800')

LSF_matrix = Korg.compute_LSF_matrix(
    synthesis_wls,
    spectrum_s['waveobs'],
    28_000)

In [11]:
windows = dict()

# Oxygen
windows[8] = [(Korg.air_to_vacuum(7771.359),Korg.air_to_vacuum(7772.509)),(Korg.air_to_vacuum(7773.522),Korg.air_to_vacuum(7774.782)),(Korg.air_to_vacuum(7774.912),Korg.air_to_vacuum(7775.962))]

# # Na
# windows[11] = [(Korg.air_to_vacuum(4751.689),Korg.air_to_vacuum(4751.944)),(Korg.air_to_vacuum(5682.517),Korg.air_to_vacuum(5682.997)),(Korg.air_to_vacuum(5687.917),Korg.air_to_vacuum(5688.392))]

In [ ]:
element = "O"

params = dict(O = 0.0)

fit_result = Korg.Fit.fit_spectrum(
    spectrum_s['waveobs'],
    spectrum_s['flux'],
    spectrum_s['err'],
    galah_lines, 
    params,
    stellar_parameters,
    windows=windows[Korg.atomic_numbers[element]],
    synthesis_wls=synthesis_wls, LSF_matrix=LSF_matrix
)

In [ ]:
plt.plot(
    spectrum_s['waveobs'],
    spectrum_s['flux']
)
plt.plot(
    spectrum_s['waveobs'][fit_result.obs_wl_mask],
    fit_result.best_fit_flux
)